## HDP Model with Genesim
### 1. Corpus Creation

In [3]:
# Retrieve data from Preprocessing
%store -r data_lemmatized df

from gensim import corpora, models
from pprint import pprint
import gensim

# Create Dictionary
id2word = gensim.corpora.Dictionary(data_lemmatized)

# Filtering out of tokens appearing in less than 20 documents or more than 70% of documents
id2word.filter_extremes(no_below=20, no_above=0.7)

# Create Corpus
bow_corpus = [id2word.doc2bow(doc) for doc in data_lemmatized]

# Create the TF-IDF model based on the bag-of-words corpus
tfidf_model = models.TfidfModel(bow_corpus)

# Apply the TF-IDF transformation to the corpus
tfidf_corpus = tfidf_model[bow_corpus]

for doc in tfidf_corpus:
    pprint(doc)
    break

[(0, 0.17382729590908783),
 (1, 0.1288014874967858),
 (2, 0.1412357432076524),
 (3, 0.15133198050925556),
 (4, 0.1936206866693167),
 (5, 0.19747385914846383),
 (6, 0.26781367503121173),
 (7, 0.22566439690651724),
 (8, 0.18752474079225756),
 (9, 0.33085597454994464),
 (10, 0.33190694735579895),
 (11, 0.1029348335730885),
 (12, 0.18205492681233307),
 (13, 0.0474871632327675),
 (14, 0.1018887534119497),
 (15, 0.36581431612253107),
 (16, 0.35671136727114133),
 (17, 0.09775843842261633),
 (18, 0.21368871501557465),
 (19, 0.14420839522893883),
 (20, 0.13348631927690002),
 (21, 0.21247390946953187)]


### 2. Defauld HDP model

In [4]:
# calculate hdp model with default parameters
hdp_model = HdpModel(tfidf_corpus, id2word)

In [4]:
# show initial topics
hdp_model.show_topics()

[(0,
  '0.003*opportunity + 0.002*cause + 0.002*marketing + 0.002*fast_track + 0.002*course + 0.002*cold + 0.002*shorter_line + 0.002*crappy + 0.002*refresh + 0.002*thoroughly_enjoye + 0.002*recall + 0.002*exhaust + 0.002*prepared + 0.002*reply + 0.002*dining + 0.002*anxious + 0.002*max_pass + 0.002*amazed + 0.002*sweep + 0.002*month'),
 (1,
  '0.003*spite + 0.003*epcot + 0.002*important + 0.002*annual_passe + 0.002*international + 0.002*profit + 0.002*universal_studio + 0.002*stake + 0.002*san_diego + 0.002*gate + 0.002*iphone + 0.002*stuck + 0.002*odd + 0.002*lead + 0.002*clean_safe + 0.002*well_traine + 0.002*organization + 0.002*public + 0.002*submarine_voyage + 0.002*rent'),
 (2,
  '0.002*directly + 0.002*least_twice + 0.002*sweater + 0.002*supply + 0.002*truth + 0.002*forget + 0.002*photo + 0.002*bread + 0.002*really_enjoye + 0.002*heat + 0.002*info + 0.002*plenty + 0.002*entrance_fee + 0.002*cartoon_character + 0.002*space_mountain + 0.002*tea + 0.002*adore + 0.002*ticket + 0.00

#### Default coherence score

In [5]:
from gensim.models import CoherenceModel

# Compute Coherence Score with default alpha and beta values
coherence_model = CoherenceModel(model=hdp_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_score = coherence_model.get_coherence()
print('Coherence Score: ', coherence_score)

Coherence Score:  0.5338892810010523


### 3. Compute model performance metrics
#### Calculation of Coherence Score with varying Truncation Level

In [18]:
# Define function for calculation of coherence values
def compute_coherence_values(corpus, dictionary, model):
    coherence_model = CoherenceModel(model=model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    return coherence_model.get_coherence()

In [23]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
from gensim.models import HdpModel
import pandas as pd

# Create list with varying truncation levels between 5 and 50
t_list = [t for t in range(5, 50, 5)]
results = []

# Define funtion for training hdp model with varying truncation level
def train_hdp_model_t(t):
    hdp_model = HdpModel(corpus=tfidf_corpus, id2word=id2word, T=t)
    
    # Compute coherence value for the trained model
    cv = compute_coherence_values(corpus=tfidf_corpus, dictionary=id2word, model=hdp_model)
    
    # Get the number of topics generated by the model
    num_topics = len(hdp_model.get_topics())
    
    # Store the result in a dictionary
    result = {'Truncation Level': t, 'Coherence': cv, 'Num_Topics': num_topics}
    
    return result

# Create a progress bar and a thread pool executor for concurrent execution
with tqdm(total=len(t_list), desc="Training HDP Models") as pbar, ThreadPoolExecutor() as executor:
    # Submit tasks to the executor for each truncation level in t_list
    futures = [executor.submit(train_hdp_model_t, t) for t in t_list]
    
    # Wait for the tasks to complete and retrieve the results
    for future in as_completed(futures):
        result = future.result()
        results.append(result)
        pbar.update(1)

# Create a pandas DataFrame from the results
df2 = pd.DataFrame(results)

# Sort the DataFrame based on the 'Coherence' column in descending order
df2.sort_values('Coherence', ascending=False)


Training HDP Models: 100%|██████████| 9/9 [03:59<00:00, 26.61s/it] 


,Truncation Level,Coherence,Num_Topics
0,10,0.536437,10
1,5,0.499534,5
2,15,0.525538,15
3,20,0.521723,20
4,25,0.530027,25
5,30,0.552913,30
6,40,0.534547,40
7,35,0.537099,35
8,45,0.526046,45


#### Calculation of Coherence Score with varying alpha, beta & gamma

In [26]:
# Define function for training the model with different parameters and predefined truncation level
def train_hdp_model_params(alpha, beta, gamma):
    hdp_model = HdpModel(corpus=tfidf_corpus, id2word=id2word, alpha=alpha, eta=beta, gamma=gamma, T=30)
    
    # Compute coherence value for the trained model
    cv = compute_coherence_values(corpus=tfidf_corpus, dictionary=id2word, model=hdp_model)
    
    # Get the number of topics generated by the model
    num_topics = len(hdp_model.get_topics())
    return cv, num_topics

# define range for alpha, beta & gamma
alpha_list = [0.01, 0.31, 0.61, 0.91]
beta_list = [0.01, 0.31, 0.61, 0.91]
gamma_list = [1,2,3]

results = []


# Iterate over alpha and gamma values
for alpha in alpha_list:
    for beta in beta_list:
        for gamma in gamma_list:
            # Train the HDP model
            cv, num_topics = train_hdp_model_30(alpha, beta, gamma)

            # Store the results
            result = {'Alpha': alpha, 'Beta': beta, 'Gamma': gamma, 'Coherence': cv, 'Num_Topics': num_topics}
            results.append(result)
        
# create data frame with results                
df3 = pd.DataFrame(results)
df3.to_csv('hdp_tuning_results_gensim.csv', index=False)

In [27]:
# Read the model_results from the CSV file
model_results = pd.read_csv('hdp_tuning_results_gensim.csv')

# display the metrics and coherence score
metrics_df = model_results.sort_values('Coherence', ascending=False).head()   
metrics_df

,Alpha,Beta,Gamma,Coherence,Num_Topics
22,0.31,0.91,2,0.548468,30
28,0.61,0.31,2,0.544316,30
6,0.01,0.61,1,0.542283,30
16,0.31,0.31,2,0.542131,30
18,0.31,0.61,1,0.541652,30


#### Selection of the optimum alpha and beta values

In [32]:
# select alpha, beta & gamma with the highest coherence value from the dataframe
alpha = metrics_df.iloc[0,1]
beta = metrics_df.iloc[0,2]
gamma = metrics_df.iloc[0,3]

print(alpha, beta, gamma)
    

# calculate the hdp with the selected parameters
best_hdp_model = HdpModel(corpus=tfidf_corpus, id2word=id2word, alpha=alpha, eta=beta, gamma=gamma, T=30)

0.91 2 0.5484679966807385


#### Topics distribution across documents

In [57]:
import numpy as np

# Define function for extracting the topic probabilities and weights from the HDP model
def topic_prob_extractor(gensim_hdp):   
    shown_topics = gensim_hdp.show_topics(num_topics=30, formatted=False)
    topics_nos = [x[0] for x in shown_topics]
    weights = [sum([item[1] for item in x[1]]) for x in shown_topics]

    return pd.DataFrame({'topic_id': topics_nos, 'weight': weights})

# Retrieve the document-topic assignments
topic_assignments = best_hdp_model[tfidf_corpus]

# Count the number of documents assigned to each topic
topic_counts = {}
for doc_topics in topic_assignments:
    for topic_id, topic_prob in doc_topics:
        topic_counts[topic_id] = topic_counts.get(topic_id, 0) + 1

# Create the DataFrame with topic counts and weights
df_document_topic = pd.DataFrame({'Num Documents': topic_counts}).sort_values(by='Num Documents', ascending=False)

# Calculate the topic weights
df_topic_weights = topic_prob_extractor(best_hdp_model)

# Merge the topic weights into the DataFrame
df_document_topic = df_document_topic.merge(df_topic_weights, left_index=True, right_on='topic_id')
df_document_topic = df_document_topic[['topic_id', 'Num Documents', 'weight']]

# Sort the DataFrame by weights in descending order
df_document_topic = df_document_topic.sort_values(by='weight', ascending=False)

# Calculate the topic distribution
df_topic_distribution = df_document_topic['topic_id'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']

# Display the DataFrame
df_document_topic

,topic_id,Num Documents,weight
4,4,994,0.040013
5,5,784,0.038340
20,20,1244,0.038269
25,25,868,0.038178
1,1,914,0.038053
19,19,1332,0.038043
8,8,1213,0.037732
16,16,1188,0.037638
24,24,977,0.037633
12,12,1093,0.037386


#### Intertopic Distance Map

In [59]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import warnings

# disable deprecation warning
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Visualize the topics
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(best_hdp_model, tfidf_corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
3     -0.001262  0.000491       1        1  6.231362
27     0.003493 -0.011284       2        1  6.134505
2     -0.005408 -0.038449       3        1  4.232243
28     0.024716  0.019806       4        1  4.218540
19    -0.012068  0.001760       5        1  4.201861
0      0.023983 -0.011137       6        1  3.974485
20    -0.012816  0.034140       7        1  3.947762
8      0.025076  0.014738       8        1  3.765706
13     0.028703 -0.003858       9        1  3.614066
16    -0.016737 -0.016709      10        1  3.425087
26    -0.010993  0.018088      11        1  3.336947
10    -0.040175  0.007666      12        1  3.283732
21     0.001559 -0.012288      13        1  3.220671
29     0.010381  0.022465      14        1  3.213701
12     0.003794 -0.001262      15        1  3.207321
7      0.006904 -0.030202      16        1  3.152149
22     0.008141 -0.000257      17        1  3.023945
24    -0.036114 -0.000569      18        1  2.999485
15    -0.022543  0.001422      19        1  2.973706
4      0.033126 -0.036018      20        1  2.905078
23     0.005991 -0.014413      21        1  2.868355
6      0.013217  0.022822      22        1  2.796542
1     -0.017496  0.053559      23        1  2.697151
17     0.003373 -0.005656      24        1  2.663085
9      0.017507 -0.012845      25        1  2.642625
25     0.038322  0.016809      26        1  2.505966
11     0.006881  0.015947      27        1  2.453419
5     -0.043006 -0.036580      28        1  2.274373
14    -0.013352 -0.007782      29        1  2.248974
18    -0.023195  0.009595      30        1  1.787158, topic_info=                   Term       Freq      Total Category  logprob  loglift
1691  cartoon_character  38.000000  38.000000  Default  30.0000  30.0000
1213       shorter_line  33.000000  33.000000  Default  29.0000  29.0000
2602       ridiculously  37.000000  37.000000  Default  28.0000  28.0000
541             anaheim  48.000000  48.000000  Default  27.0000  27.0000
1179     character_dine  42.000000  42.000000  Default  26.0000  26.0000
...                 ...        ...        ...      ...      ...      ...
2754         projection   2.350142  32.979606  Topic30  -6.5510   1.3831
2551            villain   2.544441  42.157697  Topic30  -6.4716   1.2170
180                 few   2.501568  43.517345  Topic30  -6.4886   1.1683
1917            noticed   2.287431  34.820442  Topic30  -6.5781   1.3018
290           delicious   2.256371  31.560026  Topic30  -6.5917   1.3864

[1263 rows x 6 columns], token_table=      Topic      Freq Term
term                      
2774      1  0.051258  aaa
2774      2  0.051258  aaa
2774      3  0.025629  aaa
2774      4  0.025629  aaa
2774      5  0.025629  aaa
...     ...       ...  ...
1764     24  0.025060  zoo
1764     25  0.025060  zoo
1764     26  0.025060  zoo
1764     27  0.050120  zoo
1764     28  0.025060  zoo

[20784 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 28, 3, 29, 20, 1, 21, 9, 14, 17, 27, 11, 22, 30, 13, 8, 23, 25, 16, 5, 24, 7, 2, 18, 10, 26, 12, 6, 15, 19])

### 4. Final Results
#### Top 10 words assigned to each topic

In [36]:
# Show top 10 keywords for each topic
def show_topics(hdp_model, n_words=20):
    topic_keywords = []
    for topic_weights in hdp_model.get_topics():
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append([id2word[idx] for idx in top_keyword_locs])
    return topic_keywords

topic_keywords = show_topics(hdp_model=best_hdp_model, n_words=10)        

# Topic-Keywords DataFrame
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i+1) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i+1) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10
Topic 1,cartoon_character,nap,cheese,unfortunate,sore,young,occasion,approximately,product,middle
Topic 2,inexpensive,wifi,virtually,super_busy,lack,anaheim,treasure,first_timer,unforgettable,advance
Topic 3,reaction,farm,hop,serious,keep,organisation,non,loud,dark,water
Topic 4,outlet,four_day,wanted,paint,relaxed,racer,better_value,visit,knowledge,least
Topic 5,disaster,character_dine,prevent,success,walk_around,sea_world,blue,solo,concept,french_market
Topic 6,accord,extremely_busy,aware,prepared,absolutely,grow,transfer,snack,somewhere_else,pricing
Topic 7,improve,view,ridiculously,oppose,confuse,universal,half,every_detail,split,exhausting
Topic 8,triple,multi_day,color_show,grumpy,senior,resort,worked_well,management,step,really_enjoyed
Topic 9,gift_shop,begin,relive,assist,win,popular_ride,special,old,easter,space
Topic 10,literally,reason,attack,came_across,shove,local,useless,long_period,comparison,changed_since


#### Assignment of the reviews to the topics

In [58]:
# Set column width to maximum to see the whole review
pd.set_option('display.max_colwidth', None)

# Reset index of the DataFrame for matching
df = df.reset_index(drop=True)
df_document_topic = df_document_topic.reset_index(drop=True)

# Join the dataset with the 'topic_id' column from df_document_topic
df_joined = pd.merge(df, df_document_topic['topic_id'], left_index=True, right_index=True)

df_joined.head(10)

,Review_Text,topic_id
0,"this place has always been and forever will be special. the feeling you get entering the park, seeing the characters and different attractions is just priceless. this is definitely a dream trip for all ages, especially young kids. spend the money and go to disneyland, you will not regret it",4
1,"a great day of simple fun and thrills. bring cash, nothing is cheap, but we knew that it's disney. but they are great letting you bring in your own food, drinks, etc but read the list closely, we list several items at the entry gates (selfy sticks, glass refill bottles, etc). it is worth buying the photo pass and fastpass. have fun!",5
2,all and all a great day was had. the crowds are huge and ride times sometimes up to a 50 min wait but worth it. very disappointed the castle was under repair and covered but we understood. if you want to take a short cut on the rides grab a max pass for 15 each it allows you to book a time every 90mins and believe me it's a great feeling floating past the ques.,20
3,"having been to the florida location numerous times over the years i didn't know how this one would compare. the much smaller footprint made it quicker easier to get from ride to ride. the cinderella's castle was much much smaller, but also being renovated. many of the rides were identical, while some were notably different. the indiana jones ride (which fl does not have) was awesome. space mountain was much more puke worthy. it's a small world was way more beautiful from the exterior. they had several of the old rides that fl doesn't have (like pinocchio and mr. toad). i love how their fast passes worked... it was easy to use and we were able to use them back to back on rides, fast passing almost everything we wanted to ride. we went the first week of april and while crowded, i didn't think it was too bad. we were only in the park for like 7 hours and i do wish i'd had about 3 more hours to do everything.",25
4,"had the 4 day pass, spent 3 at dl and one at ca. great place to visit. we will be back. saw a lot in those days, but there is more that we missed, and more we want to see again.",1
5,oh my god you can really forget your self and enjoy everything you face there its a huge word of fun,19
6,"we were so sad that the castle was under refurbishment, but we didn t let it ruin our good time. i purchased the maxpass before we went and it was super easy to use and very convenient. maxpass includes free pictures taken by disney photographers. i have been to disney world and there is no shortage of photographers. this was not the case here. they definitely need to add more. we were able to do all of the rides that we wanted to do, and we got the snacks that we wanted to try. we spent two days here and on the last day we watched the fireworks show. it was amazing. all of the workers were polite and helpful.",8
7,"took my son to the park on a weekend(fri sun). very busy all over the places. my son enjoyed it, but he was very sad that he could not try a lot of attractions as the wait time was huge.....( ranging from 60 90 minutes) if you have a lot of time to waste waiting in the line for the attractions, fastpass system at the park is just a ripoff. you can have only one attraction added to your plan at any point, once you complete that attraction you can add one more.",16
8,"there's nothing like disneyland and it should be on your bucket list if you've never been. park is well maintained and parking is ok, but not cheap at $25. tickets can cost up to $150 per adult (10 years old and up) during peak periods. food is not cheap either. if you take a family without a season's pass, you could easily spend hundreds of dollars just on admission tickets. if you go frequently, get a disney credit card to get free park perks and park promotions. downtown disney is also a fun place to visit.",24
9,"i think it s because going to disneyland is one of my earliest memories, but i just love visiting the park every few years. you won t